# En este Notebook realizaremos detección, censura y remoción de atípicos utilizando los métodos de RIQ, media-desviación estándar, cuantiles y LOF.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerPathCollection
from sklearn.neighbors import LocalOutlierFactor

import sys
sys.path.append('../../src/visualization/')
import diagnostic_functions

In [ ]:
# Cargando datos Boston House
boston_dataset = pd.read_csv('../../datasets/raw/boston.csv')

In [ ]:
# Creamos un dataframe con variables independientes. Usaremos solo 3 variables para propósitos de demostración
#boston = boston_dataset.filter(['RM', 'LSTAT', 'CRIM'], axis=1)
boston = boston_dataset[['RM', 'LSTAT', 'CRIM']].copy()

boston.head()

### Visualización

In [ ]:
# Visualizamos con la función de diagnóstico los valores extremos en RM
diagnostic_functions.diagnostic_plots2(boston, 'RM')


In [ ]:


def find_boundaries_RIQ(df, variable, distance):

    # Calculamos las froteras (distribución sesgada)
    # La distancia es pasada como argunmento

    IQR = df[variable].quantile(0.75) - df[variable].quantile(0.25)

    lower_boundary = df[variable].quantile(0.25) - (IQR * distance)
    upper_boundary = df[variable].quantile(0.75) + (IQR * distance)

    return upper_boundary, lower_boundary



def find_boundaries_zscore(df, variable, distance):
    lower_boundary = df[variable].mean() - (df[variable].std() * distance)
    upper_boundary = df[variable].mean() + (df[variable].std() * distance)
    return upper_boundary, lower_boundary

def find_boundaries_quantile(df, variable):
    lower_boundary = df[variable].quantile(0.05)
    upper_boundary = df[variable].quantile(0.95)
    return upper_boundary, lower_boundary

In [ ]:
# Encontremos los límites RM usando la regla RIQ
RM_upper_limit, RM_lower_limit = find_boundaries_RIQ(boston, 'RM',1.5)
RM_upper_limit, RM_lower_limit

### Censura de valores atípicos

In [ ]:
# Censuremos los outliers por los limites máximo y mínimo

boston['RM_censured']= np.where(boston['RM'] > RM_upper_limit, RM_upper_limit,
                       np.where(boston['RM'] < RM_lower_limit, RM_lower_limit, boston['RM']))

boston.head(10)

In [ ]:
diagnostic_functions.diagnostic_plots2(boston,'RM_censured')

### Remoción de valores atípicos

In [ ]:
# Marquemos los outliers en el cto. de datos

outliers_RM = np.where(boston['RM'] > RM_upper_limit, True,
                       np.where(boston['RM'] < RM_lower_limit, True, False))
outliers_RM

In [ ]:
# Removemos del conjunto de datos solo los outliers en RM

boston_trimmed = boston.loc[~outliers_RM ]
boston.shape, boston_trimmed.shape

In [ ]:
# Exploremos los valores extremos en los datos removidos
# para la variable RM vemos muchos menos outliers 
# que en los datos originales
diagnostic_functions.diagnostic_plots2(boston_trimmed, 'RM')

### Detección y remociuón de atípicos utilizando LOF

In [ ]:
np.random.seed(42)

# Generamos los datos sintéticos (inliers) de entrenamiento para propositos de demostración
X_inliers = 0.3 * np.random.randn(100, 2)
X_inliers = np.r_[X_inliers + 2, X_inliers - 2]

In [ ]:
# Graficamos los inliers
X_inliers.shape
plt.scatter(X_inliers[:,0],X_inliers[:,1])
plt.show()

In [ ]:
# Generemos algunos outliers (20)
X_outliers = np.random.uniform(low=-4, high=4, size=(20, 2))
X = np.r_[X_inliers, X_outliers]

In [ ]:
# Graficamos los outliers generados
X_outliers.shape
plt.scatter(X_outliers[:,0],X_outliers[:,1])
plt.show()

In [ ]:
#Ambos, inliers y outliers
X.shape
plt.scatter(X[:,0],X[:,1])
plt.show()

In [ ]:
# Creamos un narray indicando si es la observación es atípica (-1) o no (1)
n_outliers = len(X_outliers)
ground_truth = np.ones(len(X), dtype=int)
ground_truth[-n_outliers:] = -1

In [ ]:
# Instanciamos y entrenamos LOF (default) y obtenemos y_pred = 1:inlier -1:outlier
clf = LocalOutlierFactor(n_neighbors=20, contamination=0.1)
y_pred = clf.fit_predict(X)

In [ ]:
# Cuantificamos los errores
n_errors = (y_pred != ground_truth).sum()
print(n_errors)

In [ ]:
# Obtenemos los scores
X_scores = clf.negative_outlier_factor_
X_scores

In [ ]:
def update_legend_marker_size(handle, orig):
    "Customize size of the legend marker"
    handle.update_from(orig)
    handle.set_sizes([20])


plt.scatter(X[:, 0], X[:, 1], color="k", s=3.0, label="Data points")
# Graficamos circulos con radio proporcional a los outlier scores
radius = (X_scores.max() - X_scores) / (X_scores.max() - X_scores.min())
scatter = plt.scatter(
    X[:, 0],
    X[:, 1],
    s=1000 * radius,
    edgecolors="r",
    facecolors="none",
    label="Outlier scores",
)
plt.axis("tight")
plt.xlim((-5, 5))
plt.ylim((-5, 5))
plt.xlabel("prediction errors: %d" % (n_errors))
plt.legend(
    handler_map={scatter: HandlerPathCollection(update_func=update_legend_marker_size)}
)
plt.title("Local Outlier Factor (LOF)")
plt.show()

### Detección de atípicos basada en algoritmos de clustering

Se deja como ejercicio